In [ ]:
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
import os

In [ ]:
pwd

In [ ]:
ps_adata = sc.read_h5ad('ps_bulk_malignant.h5ad')

In [ ]:
ps_adata_comp = sc.read_h5ad('ps_comp_malignant.h5ad')

In [ ]:
ps_adata_comp

In [ ]:
ps_adata.obs["Cluster_Names"] = ps_adata.obs.Dataset_ID.map(dict(zip(ps_adata_comp.obs.Dataset_ID, ps_adata_comp.obs.Cluster_Names)))

In [ ]:
sc.pl.umap(ps_adata, color=['Cluster_Names', 'dpt_pseudotime', 'Malignant Cell - Mesenchymal'])

# CellRank

# Epi-High to Mes

In [ ]:
!python -m pip install cellrank

In [ ]:
import cellrank as cr

In [ ]:
pk = cr.kernels.PseudotimeKernel(ps_adata, time_key="dpt_pseudotime")
pk.compute_transition_matrix() 
print(pk)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
g2 = cr.estimators.GPCCA(pk)
print(g2)

In [ ]:
g2.compute_schur(method='brandts')
g2.plot_spectrum(real_only=True)

In [ ]:
g2.compute_macrostates(n_states=8, n_cells=20)
g2.plot_macrostates(which="all", legend_loc="right", s=100)

In [ ]:
g2.predict_initial_states(allow_overlap=True, n_cells=20, n_states=1)
g2.predict_terminal_states(allow_overlap=True, n_cells=20, n_states=1)

In [ ]:
g2.plot_macrostates(which='terminal', s=100, discrete=True)
g2.plot_macrostates(which='initial', s=100)

In [ ]:
for i in range(0,8):
    g2.plot_macrostates(which="all", legend_loc="right", states=[str(i)], s=100)

In [ ]:
g2.set_initial_states(states=['3, 6'], n_cells=20, allow_overlap=True)
g2.set_terminal_states(states=['7'], n_cells=20)

In [ ]:
g2.compute_fate_probabilities(preconditioner='ilu', solver='direct', use_petsc=False)
g2.plot_fate_probabilities(same_plot=False)

In [ ]:
g2.compute_eigendecomposition()
df = g2.compute_lineage_drivers()
df.head(10)

In [ ]:
g2.compute_eigendecomposition()
df = g2.compute_lineage_drivers()
df.head(10)

In [ ]:
ps_adata

In [ ]:
genes= df.sort_values(by='7_corr', ascending=False).head(25).index.tolist()
# genes= df[df['7_corr'] > 0.7].index.tolist()

In [ ]:
MFAP4, COL14A1, ASPN, DCN, FBN1, MGP, CXCL12, C1R, C1S, SERPING1, TCF4, HIC1, TCEAL7

In [ ]:
genes[:25]

In [ ]:
pd.DataFrame(df.sort_values(by='7_corr', ascending=False).head(100).index.tolist()).to_csv('mes_genes.csv')

In [ ]:
model = cr.models.GAM(
    ps_adata,
    n_knots=6,
    distribution="normal",    # allows negative y
    link="identity",
    grid={'lam': np.logspace(-2, 3, 10)}  # or {'lam': [10.0]}
)

In [ ]:
ps_adata

In [ ]:
import os
os.chdir('CellRank/')

In [ ]:
cr.pl.heatmap(ps_adata, model, genes=genes, time_key='dpt_pseudotime', figsize=(15,12), dpi=300, save='Epi_to_Mes_heatmap.png')

# TCGA on top 50

In [ ]:
df_tcga = pd.read_csv('df_tcga.csv', index_col='Unnamed: 0')

In [ ]:
cols_to_keep = ['submitter_id', 'time', 'event', 'gender_encoded'] + genes

In [ ]:
len(cols_to_keep)

In [ ]:
available_cols = [col for col in cols_to_keep if col in df_tcga.columns]
available_genes = [col for col in genes if col in df_tcga.columns]
df_tcga_mes = df_tcga[available_cols]

In [ ]:
df_tcga_mes

In [ ]:
from sklearn.feature_selection import VarianceThreshold
vt = VarianceThreshold(threshold=1e-6)
vt.fit(df_tcga_mes[available_genes])
mask = vt.get_support()           # e.g. [True, False, True, ...]
kept_genes = [g for g, keep in zip(available_genes, mask) if keep]
print(f"Dropped {len(available_genes) - len(kept_genes)} zero‐variance genes.")
df_tcga_mes = df_tcga_mes[kept_genes + ['gender_encoded','time','event']].copy()

In [ ]:
from lifelines import CoxPHFitter
cph = CoxPHFitter(penalizer=0.1)
cph.fit(
    df_tcga_mes,
    duration_col='time',
    event_col='event'
)

df_cph = cph.summary 

In [ ]:
df_cph.sort_values(by='p').head(20)

In [ ]:
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test

def km_plot_gene(df, gene, cut="median", ax=None):
    """Plot KM curves for gene using a binary split."""
    x = df[gene]
    if cut == "median":
        thresh = x.median()
        group = np.where(x > thresh, "High", "Low")
    else:
        # numeric threshold if you want to pass e.g., cut=2.3
        thresh = float(cut)
        group = np.where(x > thresh, "High", "Low")

    tmp = df.assign(group=group)

    kmf = KaplanMeierFitter()
    if ax is None:
        fig, ax = plt.subplots(figsize=(5,4), dpi=120)

    for label in ["High", "Low"]:
        sel = tmp["group"] == label
        kmf.fit(durations=tmp.loc[sel, "time"],
                event_observed=tmp.loc[sel, "event"],
                label=f"{label} {gene} (n={sel.sum()})")
        kmf.plot(ax=ax, ci_show=True)

    # log-rank test
    hi = tmp["group"] == "High"
    lo = tmp["group"] == "Low"
    res = logrank_test(tmp.loc[hi, "time"], tmp.loc[lo, "time"],
                       event_observed_A=tmp.loc[hi, "event"],
                       event_observed_B=tmp.loc[lo, "event"])
    p = res.p_value

    ax.set_title(f"{gene}: High vs Low (cut={thresh:.3g})")
    ax.set_xlabel("Time")
    ax.set_ylabel("Survival probability")
    ax.legend(frameon=False)
    ax.text(0.02, 0.02, f"log-rank p = {p:.3g}", transform=ax.transAxes)

    return ax, p, tmp

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(10,4), dpi=120, sharey=True)
ax, p, tmp = km_plot_gene(df_tcga, "HIC1", ax=axes)

In [ ]:
tmp.groupby('group')['time'].median()

# Epi-High to Tip

In [ ]:
ps_adata.obs.Cluster_Names = ps_adata.obs.Cluster_Names.replace('Acinar-Like', 'Epi-High')

In [ ]:
ps_adata.obs.Cluster_Names.value_counts()

In [ ]:
bulk_epi_tip = ps_adata[ps_adata.obs.Cluster_Names.isin(['Epi-High', 'Hypoxia/Senescence-High', 'EMT-Start', 'Tip'])].copy()

In [ ]:
sc.pl.umap(bulk_epi_tip, color='Cluster_Names')

In [ ]:
pk_tip = cr.kernels.PseudotimeKernel(bulk_epi_tip, time_key="dpt_pseudotime")
pk_tip.compute_transition_matrix()
print(pk_tip)

In [ ]:
g2_tip = cr.estimators.GPCCA(pk_tip)
print(g2_tip)

In [ ]:
g2_tip.compute_schur(method='brandts')
g2_tip.plot_spectrum(real_only=True)

In [ ]:
g2_tip.compute_macrostates(n_states=8, n_cells=10)
g2_tip.plot_macrostates(which="all", legend_loc="right", s=100)

In [ ]:
for i in range(0,8):
    g2_tip.plot_macrostates(which="all", legend_loc="right", states=[str(i)], s=100)

In [ ]:
g2_tip.set_initial_states(states=['3', '4'], n_cells=10, allow_overlap=True)
g2_tip.set_terminal_states(states=['5'], n_cells=8, allow_overlap=True)

In [ ]:
g2_tip.plot_macrostates(which='initial', s=100)
g2_tip.plot_macrostates(which='terminal', s=100)

In [ ]:
g2_tip.compute_fate_probabilities(preconditioner='ilu', solver='direct', use_petsc=False)
g2_tip.plot_fate_probabilities(same_plot=False)

In [ ]:
g2_tip.compute_eigendecomposition()
df_tip = g2_tip.compute_lineage_drivers()
df_tip.head(10)

In [ ]:
genes_tip = df_tip.sort_values(by='5_corr', ascending=False).head(25).index.tolist()

In [ ]:
genes_tip

In [ ]:
pd.DataFrame(df_tip.sort_values(by='5_corr', ascending=False).head(100).index.tolist()).to_csv('tip_genes.csv')

In [ ]:
model = cr.models.GAM(bulk_epi_tip,  n_knots=6,
    distribution="normal",    # allows negative y
    link="identity",
    grid={'lam': np.logspace(-2, 3, 10)}  # or {'lam': [10.0]}
)

In [ ]:
cr.pl.heatmap(bulk_epi_tip, model, genes=genes_tip, time_key='dpt_pseudotime', figsize=(15,12), dpi=300, save='Epi_to_Tip_heatmap.png')

# TCGA

In [ ]:
cols_to_keep = ['submitter_id', 'time', 'event', 'gender_encoded'] + genes_tip

In [ ]:
len(cols_to_keep)

In [ ]:
available_cols = [col for col in cols_to_keep if col in df_tcga.columns]
available_genes = [col for col in genes_tip if col in df_tcga.columns]
df_tcga_mes = df_tcga[available_cols]

In [ ]:
df_tcga_mes

In [ ]:
from sklearn.feature_selection import VarianceThreshold
vt = VarianceThreshold(threshold=1e-6)
vt.fit(df_tcga_mes[available_genes])
mask = vt.get_support()           # e.g. [True, False, True, ...]
kept_genes = [g for g, keep in zip(available_genes, mask) if keep]
print(f"Dropped {len(available_genes) - len(kept_genes)} zero‐variance genes.")
df_tcga_mes = df_tcga_mes[kept_genes + ['gender_encoded','time','event']].copy()

In [ ]:
from lifelines import CoxPHFitter
cph = CoxPHFitter(penalizer=0.1)
cph.fit(
    df_tcga_mes,
    duration_col='time',
    event_col='event'
)

df_cph = cph.summary 

In [ ]:
df_cph.sort_values(by='p').head(20)

# DGE between radio tip vs all others
# only malginant cells
# dotplot for radio specific and TCGA

In [ ]:
ps_adata

In [ ]:
sc.pl.umap(ps_adata, color='Cluster_Names')

In [ ]:
sc.tl.rank_genes_groups(ps_adata, groupby='Cluster_Names', groups=['Tip'])

In [ ]:
sc.pl.rank_genes_groups(ps_adata)

In [ ]:
df_dge_tip = pd.DataFrame(ps_adata.uns['rank_genes_groups']['names'])

In [ ]:
df_dge_tip

In [ ]:
genes = df_dge_tip['Tip'].head(50).tolist()

In [ ]:
df_tcga = pd.read_csv('df_tcga.csv', index_col='Unnamed: 0')

In [ ]:
cols_to_keep = ['submitter_id', 'time', 'event', 'gender_encoded'] + genes

In [ ]:
len(cols_to_keep)

In [ ]:
available_cols = [col for col in cols_to_keep if col in df_tcga.columns]
available_genes = [col for col in genes if col in df_tcga.columns]
df_tcga_mes = df_tcga[available_cols]

In [ ]:
df_tcga_mes

In [ ]:
from sklearn.feature_selection import VarianceThreshold
vt = VarianceThreshold(threshold=1e-6)
vt.fit(df_tcga_mes[available_genes])
mask = vt.get_support()           # e.g. [True, False, True, ...]
kept_genes = [g for g, keep in zip(available_genes, mask) if keep]
print(f"Dropped {len(available_genes) - len(kept_genes)} zero‐variance genes.")
df_tcga_mes = df_tcga_mes[kept_genes + ['gender_encoded','time','event']].copy()

In [ ]:
from lifelines import CoxPHFitter
cph = CoxPHFitter(penalizer=0.1)
cph.fit(
    df_tcga_mes,
    duration_col='time',
    event_col='event'
)

df_cph = cph.summary 

In [ ]:
df_cph.sort_values(by='p').head(20)

In [ ]:
sc.pl.matrixplot(ps_adata, groupby='Cluster_Names', var_names=genes, standard_scale='var')

In [ ]:
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test

def km_plot_gene(df, gene, cut="median", ax=None):
    """Plot KM curves for gene using a binary split."""
    x = df[gene]
    if cut == "median":
        thresh = x.median()
        group = np.where(x > thresh, "High", "Low")
    else:
        # numeric threshold if you want to pass e.g., cut=2.3
        thresh = float(cut)
        group = np.where(x > thresh, "High", "Low")

    tmp = df.assign(group=group)

    kmf = KaplanMeierFitter()
    if ax is None:
        fig, ax = plt.subplots(figsize=(5,4), dpi=120)

    for label in ["High", "Low"]:
        sel = tmp["group"] == label
        kmf.fit(durations=tmp.loc[sel, "time"],
                event_observed=tmp.loc[sel, "event"],
                label=f"{label} {gene} (n={sel.sum()})")
        kmf.plot(ax=ax, ci_show=True)

    # log-rank test
    hi = tmp["group"] == "High"
    lo = tmp["group"] == "Low"
    res = logrank_test(tmp.loc[hi, "time"], tmp.loc[lo, "time"],
                       event_observed_A=tmp.loc[hi, "event"],
                       event_observed_B=tmp.loc[lo, "event"])
    p = res.p_value

    ax.set_title(f"{gene}: High vs Low (cut={thresh:.3g})")
    ax.set_xlabel("Time")
    ax.set_ylabel("Survival probability")
    ax.legend(frameon=False)
    ax.text(0.02, 0.02, f"log-rank p = {p:.3g}", transform=ax.transAxes)

    return ax, p, tmp

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(10,4), dpi=120, sharey=True)
ax, p, tmp = km_plot_gene(df_tcga, "ROR2", ax=axes)

In [ ]:
mal_cols = ps_adata.obs.columns[ps_adata.obs.columns.str.contains('Mal')].tolist()

In [ ]:
agg

In [ ]:
agg = (
    ps_adata.obs
      .groupby("Cluster_Names")[mal_cols]
      .median()
      .sort_index()
).sort_values(by='Malignant Cell - EMT', ascending=False)

# 2) Grouped bar plot with matplotlib
clusters = agg.index.to_list()
x = np.arange(len(clusters))
n_features = len(mal_cols)
group_width = 0.8
bar_width = group_width / n_features

fig, ax = plt.subplots(figsize=(max(12, len(clusters)*0.8), 6))

for i, col in enumerate(mal_cols):
    ax.bar(
        x - group_width/2 + i*bar_width + bar_width/2,
        agg[col].values,
        width=bar_width,
        label=col
    )

ax.set_xticks(x)
ax.set_xticklabels(clusters, rotation=45, ha='right')
ax.set_xlabel("Cluster")
ax.set_ylabel("Median (%)")
ax.set_title("Median malignant features by cluster")
ax.legend(ncols=2, fontsize=8)
fig.tight_layout()
plt.show()

# DGE for only radio samples of Tip vs others

In [ ]:
ps_adata.obs.head()

In [ ]:
ps_adata[((ps_adata.obs.Cluster_Names == 'Tip') & (ps_adata.obs.TreatmentType.str.contains('Radio')))].obs

In [ ]:
ps_adata.obs['Radio_Tip'] = np.where(((ps_adata.obs.Cluster_Names == 'Tip') & (ps_adata.obs.TreatmentType.str.contains('Radio'))), 'Radio', 'Others')

In [ ]:
ps_adata.obs.Radio_Tip.value_counts()

In [ ]:
sc.tl.rank_genes_groups(ps_adata, groupby='Radio_Tip')

In [ ]:
sc.pl.rank_genes_groups(ps_adata)

In [ ]:
df_dge_tip_radio = pd.DataFrame(ps_adata.uns['rank_genes_groups']['names']['Radio'])

In [ ]:
df_dge_tip_radio

In [ ]:
genes = df_dge_tip_radio[0].head(100).tolist()

In [ ]:
cols_to_keep = ['submitter_id', 'time', 'event', 'gender_encoded'] + genes

In [ ]:
len(cols_to_keep)

In [ ]:
available_cols = [col for col in cols_to_keep if col in df_tcga.columns]
available_genes = [col for col in genes if col in df_tcga.columns]
df_tcga_mes = df_tcga[available_cols]

In [ ]:
df_tcga_mes

In [ ]:
from sklearn.feature_selection import VarianceThreshold
vt = VarianceThreshold(threshold=1e-6)
vt.fit(df_tcga_mes[available_genes])
mask = vt.get_support()           # e.g. [True, False, True, ...]
kept_genes = [g for g, keep in zip(available_genes, mask) if keep]
print(f"Dropped {len(available_genes) - len(kept_genes)} zero‐variance genes.")
df_tcga_mes = df_tcga_mes[kept_genes + ['gender_encoded','time','event']].copy()

In [ ]:
from lifelines import CoxPHFitter
cph = CoxPHFitter(penalizer=0.1)
cph.fit(
    df_tcga_mes,
    duration_col='time',
    event_col='event'
)

df_cph = cph.summary 

In [ ]:
df_cph.sort_values(by='p').head(20)

In [ ]:
# sc.pl.matrixplot(ps_adata, groupby='Radio_Tip', var_names=genes, standard_scale='var')

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(10,4), dpi=120, sharey=True)
ax, p, tmp = km_plot_gene(df_tcga, 'TSPYL2', ax=axes)

# group 20 genes

In [ ]:
from sklearn.preprocessing import StandardScaler
from lifelines import CoxPHFitter

In [ ]:
genes = df_dge_tip_radio[0].head(20).tolist()

present = [g for g in genes if g in df_tcga.columns]
Z = pd.DataFrame(StandardScaler().fit_transform(df_tcga[present]),
                 columns=present, index=df_tcga.index)
df_tcga["Radio_Tip_20_score"] = Z.median(axis=1)

covars = ["Radio_Tip_20_score", "gender_encoded"]
cph = CoxPHFitter()
cph.fit(df_tcga[["time","event"]+covars], duration_col="time", event_col="event")
print(cph.summary.loc["Radio_Tip_20_score", ["coef","exp(coef)","p","coef lower 95%","coef upper 95%"]])

In [ ]:
df_tcga